: 

In [3]:
%pip install ipykernel mediapipe numpy tqdm ffmpeg ffprobe


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


BASIC ONE


[notice] A new release of pip is available: 24.1.1 -> 25.3
[notice] To update, run: C:\Users\satvi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tqdm import tqdm
import subprocess
import json
import os

# --- 1. SETUP & HELPER FUNCTIONS ---

mp_pose = mp.solutions.pose

def draw_body_landmarks(image, landmarks):
    """Draws the body skeleton on an image."""                                              
    h, w, _ = image.shape
    keypoints = [
        mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER,
        mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.RIGHT_ELBOW,
        mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.RIGHT_WRIST,
        mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP,
        mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.RIGHT_KNEE,
        mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.RIGHT_ANKLE,
    ]
    for lm_enum in keypoints:
        lm = landmarks.landmark[lm_enum.value]
        cv2.circle(image, (int(lm.x * w), int(lm.y * h)), 5, (0, 255, 0), -1)

    pairs = [
        (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),
        (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP),
        (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP),
        (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP),
        (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW),
        (mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
        (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW),
        (mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST),
        (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE),
        (mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE),
        (mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE),
        (mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE),
    ]
    for a, b in pairs:
        pa, pb = landmarks.landmark[a.value], landmarks.landmark[b.value]
        cv2.line(image, (int(pa.x * w), int(pa.y * h)), (int(pb.x * w), int(pb.y * h)), (255, 0, 0), 2)
    return image


def get_video_resolution(video_path):
    """Gets video resolution using ffprobe."""
    command = ['ffprobe', '-v', 'error', '-select_streams', 'v:0',
               '-show_entries', 'stream=width,height', '-of', 'json', video_path]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if result.returncode == 0:
        info = json.loads(result.stdout)
        return info['streams'][0]['width'], info['streams'][0]['height']
    return None, None


# --- 2. USER INPUT ---

choice = input("Enter 'v' for video upload or 'w' for webcam: ").lower().strip()

try:
    model_complexity_choice = int(input("Enter model complexity (0=light, 1=full, 2=heavy): ").strip())
    if model_complexity_choice not in [0, 1, 2]:
        print("Invalid choice. Defaulting to 1 (full).")
        model_complexity_choice = 1
except ValueError:
    print("Invalid input. Defaulting to 1 (full).")
    model_complexity_choice = 1

print(f"Using model complexity: {model_complexity_choice}")


# --- 3. VIDEO vs WEBCAM LOGIC ---

if choice == 'v':
    input_video = input("Enter the full path to your video file: ").strip()
    if not os.path.exists(input_video):
        raise FileNotFoundError(f"The file was not found at: {input_video}")

    compressed_video = "compressed_video.mp4"
    print(f"\n--- Step 2: Compressing '{input_video}' ---")

    width, height = get_video_resolution(input_video)
    if width is None:
        raise IOError("Could not read video resolution. Is ffprobe installed?")

    if height < 720:
        print("Video is below 720p. Removing audio only...")
        command = ['ffmpeg', '-i', input_video, '-c', 'copy', '-an', compressed_video, '-y']
    else:
        print("Video is 720p or higher. Resizing to 720p, removing audio, and compressing...")
        command = [
            'ffmpeg', '-i', input_video,
            '-vf', 'scale=trunc((iw/ih)*720/2)*2:720,setsar=1:1',
            '-an', '-vcodec', 'libx264', '-crf', '28', compressed_video, '-y'
        ]

    subprocess.run(command, check=True)
    print(f"✅ Compression complete. Processing '{compressed_video}'...")
    video_path = compressed_video
    use_webcam = False

elif choice == 'w':
    print("\n🎥 Using webcam.")
    video_path = 0
    use_webcam = True

else:
    raise ValueError("Invalid choice. Enter 'v' or 'w'.")


# --- 4. POSE ESTIMATION ---

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError(f"Cannot open video source: {video_path}")

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS) if not use_webcam else 30

output_filename = 'pose_output.mp4'
out = None
if not use_webcam:
    out = cv2.VideoWriter(output_filename, cv2.VideoWriter_fourcc(*'mp4v'),
                          fps, (frame_width, frame_height))

with mp_pose.Pose(
    static_image_mode=False,
    model_complexity=model_complexity_choice,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as pose:

    if use_webcam:
        print("Press 'q' to quit webcam.")
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)

            annotated = frame.copy()
            if results.pose_landmarks:
                annotated = draw_body_landmarks(annotated, results.pose_landmarks)

            cv2.imshow("MediaPipe Pose (Webcam)", annotated)
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break

    else:
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"\n--- Step 3: Running Pose Estimation on '{video_path}' ---")
        for _ in tqdm(range(frame_count)):
            ret, frame = cap.read()
            if not ret:
                break

            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)

            annotated = frame.copy()
            if results.pose_landmarks:
                annotated = draw_body_landmarks(annotated, results.pose_landmarks)

            out.write(annotated)

        print(f"\n✅ Pose estimation complete. Output saved as '{output_filename}'.")

# --- 5. CLEANUP ---
cap.release()
if out:
    out.release()
cv2.destroyAllWindows()


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Using model complexity: 0

--- Step 2: Compressing 'D:\visionpipeline\sampledata\example2.mp4' ---
Video is below 720p. Removing audio only...
✅ Compression complete. Processing 'compressed_video.mp4'...

--- Step 3: Running Pose Estimation on 'compressed_video.mp4' ---


100%|██████████| 325/325 [00:07<00:00, 41.71it/s]


✅ Pose estimation complete. Output saved as 'pose_output.mp4'.


WEB CAM WORKING

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import tqdm
import subprocess
import json
import os

# --- 1. SETUP & HELPER FUNCTIONS ---

# Create directories for compressed and output videos if they don't exist
COMPRESSED_FOLDER = "compressed videos"
OUTPUT_FOLDER = "output videos"
os.makedirs(COMPRESSED_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)


mp_pose = mp.solutions.pose

def draw_body_landmarks(image, landmarks):
    """Draws the body skeleton on an image."""
    h, w, _ = image.shape
    # List of keypoints to draw
    keypoints = [
        mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER,
        mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.RIGHT_ELBOW,
        mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.RIGHT_WRIST,
        mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP,
        mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.RIGHT_KNEE,
        mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.RIGHT_ANKLE,
    ]
    # Draw circles for each keypoint
    for lm_enum in keypoints:
        lm = landmarks.landmark[lm_enum.value]
        cv2.circle(image, (int(lm.x * w), int(lm.y * h)), 5, (0, 255, 0), -1)

    # Define connections between keypoints to form a skeleton
    pairs = [
        (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),
        (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP),
        (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP),
        (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP),
        (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW),
        (mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
        (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW),
        (mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST),
        (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE),
        (mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE),
        (mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE),
        (mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE),
    ]
    # Draw lines for each pair of connected keypoints
    for a, b in pairs:
        pa, pb = landmarks.landmark[a.value], landmarks.landmark[b.value]
        cv2.line(image, (int(pa.x * w), int(pa.y * h)), (int(pb.x * w), int(pb.y * h)), (255, 0, 0), 2)
    return image


def get_video_resolution(video_path):
    """Gets video resolution using ffprobe."""
    command = ['ffprobe', '-v', 'error', '-select_streams', 'v:0',
               '-show_entries', 'stream=width,height', '-of', 'json', video_path]
    try:
        result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
        info = json.loads(result.stdout)
        return info['streams'][0]['width'], info['streams'][0]['height']
    except (subprocess.CalledProcessError, FileNotFoundError):
        print("Error: ffprobe not found. Please ensure ffmpeg is installed and in your system's PATH.")
        return None, None


# --- 2. USER INPUT ---

choice = input("Enter 'v' for video upload or 'w' for webcam: ").lower().strip()

try:
    model_complexity_choice = int(input("Enter model complexity (0=light, 1=full, 2=heavy): ").strip())
    if model_complexity_choice not in [0, 1, 2]:
        print("Invalid choice. Defaulting to 1 (full).")
        model_complexity_choice = 1
except ValueError:
    print("Invalid input. Defaulting to 1 (full).")
    model_complexity_choice = 1

print(f"Using model complexity: {model_complexity_choice}")


# --- 3. VIDEO vs WEBCAM LOGIC ---

if choice == 'v':
    input_video = input("Enter the full path to your video file: ").strip()
    if not os.path.exists(input_video):
        raise FileNotFoundError(f"The file was not found at: {input_video}")

    # Generate organized file paths
    base_filename = os.path.basename(input_video)
    compressed_video = os.path.join(COMPRESSED_FOLDER, f"compressed_{base_filename}")
    output_filename = os.path.join(OUTPUT_FOLDER, f"output_{base_filename}")

    print(f"\n--- Step 2: Compressing '{input_video}' ---")

    width, height = get_video_resolution(input_video)
    if width is None:
        raise IOError("Could not read video resolution. Is ffprobe installed and accessible?")

    # Video compression logic
    if height < 720:
        print("Video is below 720p. Removing audio only...")
        command = ['ffmpeg', '-i', input_video, '-c', 'copy', '-an', compressed_video, '-y']
    else:
        print("Video is 720p or higher. Resizing to 720p, removing audio, and compressing...")
        command = [
            'ffmpeg', '-i', input_video,
            '-vf', 'scale=trunc((iw/ih)*720/2)*2:720,setsar=1:1',
            '-an', '-vcodec', 'libx264', '-crf', '28', compressed_video, '-y'
        ]

    subprocess.run(command, check=True)
    print(f"✅ Compression complete. Processing '{compressed_video}'...")
    video_path = compressed_video
    use_webcam = False

elif choice == 'w':
    print("\n🎥 Using webcam.")
    video_path = 0  # Use 0 for the default webcam
    use_webcam = True
    output_filename = None # No output file for webcam by default

else:
    raise ValueError("Invalid choice. Please enter 'v' or 'w'.")


# --- 4. POSE ESTIMATION ---

cap = cv2.VideoCapture(video_path,cv2.CAP_DSHOW)
if not cap.isOpened():
    raise IOError(f"Cannot open video source: {video_path}")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) if not use_webcam else 30

# Setup video writer only if processing a video file
out = None
if not use_webcam:
    out = cv2.VideoWriter(output_filename, cv2.VideoWriter_fourcc(*'mp4v'),
                          fps, (frame_width, frame_height))

with mp_pose.Pose(
    static_image_mode=False,
    model_complexity=model_complexity_choice,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as pose:

    if use_webcam:
        print("Press 'q' to quit webcam feed.")
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame from webcam.")
                break

            # Flip the frame horizontally for a mirror effect
            frame = cv2.flip(frame, 1)
            
            # Convert the BGR image to RGB
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)

            # Create a copy to draw on
            annotated_image = frame.copy()
            if results.pose_landmarks:
                annotated_image = draw_body_landmarks(annotated_image, results.pose_landmarks)

            cv2.imshow("MediaPipe Pose (Webcam)", annotated_image)
            
            # Exit loop if 'q' is pressed
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break

    else: # This block is for video file processing
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"\n--- Step 3: Running Pose Estimation on '{video_path}' ---")
        
        with tqdm(total=frame_count, desc="Processing Frames") as pbar:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # Convert the BGR image to RGB
                image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = pose.process(image_rgb)
                
                # Create a copy to draw on
                annotated_image = frame.copy()
                if results.pose_landmarks:
                    annotated_image = draw_body_landmarks(annotated_image, results.pose_landmarks)
                
                # Write the frame to the output file
                out.write(annotated_image)
                pbar.update(1)

        print(f"\n✅ Pose estimation complete. Output saved as '{output_filename}'.")

# --- 5. CLEANUP ---
cap.release()
if out:
    out.release()
cv2.destroyAllWindows()

Using model complexity: 0

🎥 Using webcam.
Press 'q' to quit webcam feed.


KeyboardInterrupt: 

In [ ]:
#training data
import cv2
import mediapipe as mp
import numpy as np
import csv
import os
import math

# --- 1. Initialization of MediaPipe Pose ---
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# --- 2. Landmark and Connection Definitions ---
# Define the landmarks to be excluded (face landmarks)
excluded_landmarks = [
    mp_pose.PoseLandmark.NOSE, mp_pose.PoseLandmark.LEFT_EYE_INNER,
    mp_pose.PoseLandmark.LEFT_EYE, mp_pose.PoseLandmark.LEFT_EYE_OUTER,
    mp_pose.PoseLandmark.RIGHT_EYE_INNER, mp_pose.PoseLandmark.RIGHT_EYE,
    mp_pose.PoseLandmark.RIGHT_EYE_OUTER, mp_pose.PoseLandmark.LEFT_EAR,
    mp_pose.PoseLandmark.RIGHT_EAR, mp_pose.PoseLandmark.MOUTH_LEFT,
    mp_pose.PoseLandmark.MOUTH_RIGHT
]

# Create a list of the 22 body landmarks we are interested in
body_landmarks_enum = [lm for lm in mp_pose.PoseLandmark if lm not in excluded_landmarks]

# Create a custom set of connections for drawing the skeleton
custom_connections = [
    connection for connection in mp_pose.POSE_CONNECTIONS
    if all(landmark not in excluded_landmarks for landmark in connection)
]


def normalize_pose_landmarks(landmarks):
    """
    Applies the full normalization pipeline to a set of pose landmarks.
    
    Args:
        landmarks: A list of landmark objects from MediaPipe.
    
    Returns:
        A flat numpy array of 44 normalized (x, y) coordinates, or None if not possible.
    """
    # 1. Convert landmarks to a NumPy array
    # Note: We only use the 22 body landmarks for our calculations and final feature vector.
    keypoints = np.array([[landmarks[lm.value].x, landmarks[lm.value].y] for lm in body_landmarks_enum])

    # --- Step 1: Filter / Clean (Implicit) ---
    # MediaPipe already provides confidence. For a robust pipeline, you would check
    # landmark.visibility here and decide how to handle low-confidence points.
    # For this script, we assume they are all valid.
    
    # --- Step 2: Translate (Center) ---
    # Use the mid-hip as the root point.
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    root_point = np.array([(left_hip.x + right_hip.x) / 2, (left_hip.y + right_hip.y) / 2])
    
    keypoints_translated = keypoints - root_point
    
    # --- Step 3: Scale ---
    # Use torso length for normalization.
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    
    mid_shoulder = np.array([(left_shoulder.x + right_shoulder.x) / 2, (left_shoulder.y + right_shoulder.y) / 2])
    # We use the translated mid-hip which is now at (0,0) after translation
    mid_hip_translated = np.array([(left_hip.x + right_hip.x) / 2, (left_hip.y + right_hip.y) / 2]) - root_point
    
    # Torso length is the distance between mid-shoulder and mid-hip
    torso_length = np.linalg.norm(mid_shoulder - root_point)
    
    # Avoid division by zero
    if torso_length < 1e-6:
        return None
        
    keypoints_scaled = keypoints_translated / torso_length
    
    # --- Step 4: Rotate ---
    # Align the body so shoulders are horizontal.
    # Get the scaled coordinates for the shoulders
    left_shoulder_scaled = keypoints_scaled[body_landmarks_enum.index(mp_pose.PoseLandmark.LEFT_SHOULDER)]
    right_shoulder_scaled = keypoints_scaled[body_landmarks_enum.index(mp_pose.PoseLandmark.RIGHT_SHOULDER)]
    
    # Calculate the angle of the shoulder line
    shoulder_angle = math.atan2(
        right_shoulder_scaled[1] - left_shoulder_scaled[1],
        right_shoulder_scaled[0] - left_shoulder_scaled[0]
    )
    
    # Create the rotation matrix for the negative angle
    rotation_angle = -shoulder_angle
    cos_a = math.cos(rotation_angle)
    sin_a = math.sin(rotation_angle)
    rotation_matrix = np.array([[cos_a, -sin_a], [sin_a, cos_a]])
    
    # Apply rotation to all keypoints
    keypoints_rotated = np.dot(keypoints_scaled, rotation_matrix.T) # Transpose for correct multiplication
    
    # --- 5. Flatten to create the final feature vector ---
    # Concatenate the [xr_i, yr_i] for all 22 joints.
    feature_vector = keypoints_rotated.flatten()
    
    return feature_vector


def process_video(video_path, exercise_name, output_csv_path):
    """
    Processes a video: extracts landmarks, normalizes them, saves to CSV, and shows annotated video.
    """
    cap = cv2.VideoCapture(video_path)
    
    # Check if the CSV file exists to write the header only once
    file_exists = os.path.isfile(output_csv_path)

    with open(output_csv_path, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)

        # Write header if the file is new
        if not file_exists:
            header = ['exercise']
            for landmark in body_landmarks_enum:
                header += [f'{landmark.name}_x', f'{landmark.name}_y']
            csv_writer.writerow(header)

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # --- MediaPipe Processing ---
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # --- Normalization and Data Saving ---
            if results.pose_landmarks:
                # Run the normalization pipeline
                normalized_landmarks = normalize_pose_landmarks(results.pose_landmarks.landmark)
                
                if normalized_landmarks is not None:
                    # Create the row for the CSV file
                    row = [exercise_name] + normalized_landmarks.tolist()
                    csv_writer.writerow(row)

                # --- Annotation ---
                # Create a copy of landmarks to prevent modification of originals
                display_landmarks = results.pose_landmarks
                # Make face landmarks invisible for drawing
                for lm_idx in excluded_landmarks:
                    display_landmarks.landmark[lm_idx.value].visibility = 0
                
                mp_drawing.draw_landmarks(
                    image,
                    display_landmarks,
                    custom_connections, # Use custom connections
                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                )

            cv2.imshow('MediaPipe Pose Annotation', image)

            if cv2.waitKey(5) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    # --- INSTRUCTIONS FOR USE ---
    # 1. Create a folder named 'videos' in the same directory.
    #    Place your exercise videos inside, e.g., 'videos/pushups_1.mp4'.
    
    # 2. Define the exercises and their corresponding video files.
    exercise_videos = {
        'pushup': ['videos/pushups_1.mp4', 'videos/pushups_2.mp4'],
        'pullup': ['videos/pullups_1.mp4'],
        'squat': ['videos/squats_1.mp4']
    }

    # 3. Specify the name of the output CSV file.
    output_csv_file = 'normalized_exercise_landmarks.csv'
    
    # --- RUN THE PROCESSING ---
    for exercise, video_list in exercise_videos.items():
        for video_path in video_list:
            if os.path.exists(video_path):
                print(f"Processing '{video_path}' for exercise: {exercise}...")
                process_video(video_path, exercise, output_csv_file)
            else:
                print(f"Warning: Video file not found at '{video_path}'")
                
    print(f"\nData processing complete. Normalized landmarks saved to '{output_csv_file}'")
    pose.close()